### 최초 한번 실행 과거 데이터 추출용 (2020.01.01  ~ 2023.01.08 데이터)

In [1]:
from selenium import webdriver as wd
# from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import chromedriver_autoinstaller

In [2]:
from bs4 import BeautifulSoup as bs
import datetime, time

import pickle
import pandas as pd
import numpy as np

import subprocess
import shutil
import pyperclip

import requests
import os, sys

In [3]:
try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제. 정보를 지울 필요가 없으면 이부분은 그냥 skip
except FileNotFoundError:
    pass

In [4]:
chrome_options = wd.ChromeOptions()
driver = wd.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

[WDM] - Downloading:  90%|█████████ | 5.94M/6.58M [00:00<00:00, 11.9MB/s]

In [5]:
while True:
    try:
    # driver.set_window_position(-10000,0) # hide windows
        main_url = 'https://kr.investing.com/'
        driver.get(main_url)
        time.sleep(1)
        break
    except:
        print("Message: unknown error: cannot determine loading status")
        print("Try Again")
        time.sleep(5)
        pass

In [6]:
try: # pop up 창이 뜨는 것을 확인
    element = driver.find_element_by_id('promoAncmtClose')
    driver.find_element_by_css_selector('#promoAncmtClose > svg > path').click()
    time.sleep(1)
    
except:
    pass

In [7]:
try: # pop up 창이 뜨는 것을 확인
    element = driver.find_element_by_id('PromoteSignUpPopUp')
    driver.find_element_by_css_selector('#PromoteSignUpPopUp > div.right > i').send_keys(Keys.ENTER)
    time.sleep(1)
    
except:
    pass

In [8]:
try: # 이미 로그인이 되어 있으면 skip
    driver.find_element_by_css_selector('#userAccount > div > a.login.bold').click() 
    time.sleep(1)
    
    id = 'kangeun@naver.com'
    pyperclip.copy(id) # control+c 기능 수행
    driver.find_element_by_id('loginFormUser_email').clear()
    driver.find_element_by_id('loginFormUser_email').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)
    
    pw = os.environ['investing_pass']
    pyperclip.copy(pw) # control+c 기능 수행
    driver.find_element_by_id('loginForm_password').clear()
    driver.find_element_by_id('loginForm_password').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)  
    
    driver.find_element_by_css_selector('#signup > a.newButton.orange').click()  
    time.sleep(2)
    
except:
    pass

In [9]:
def correct_date_format(df):
# 시간, 미국, 한국 접속사이트에 따라 attribute 가 변경되서 나타나기 때문에 error  처리를 위해 try 사용
    try:
        df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%b %d, %Y"))
    except:
        try:
            df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%m/%d/%Y"))
        except:
            try:
                df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y- %m- %d"))
            except:
                df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y년 %m월 %d일"))
    
    return df

In [10]:
def get_data(url, column):
    driver.get(url)
    input("임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요")

    try :
        df = pd.read_html(driver.page_source, attrs={"id": "curr_table"}, flavor=["lxml", "bs4"])[0]
    except:
        df = pd.read_html(driver.page_source, attrs={"data-test": "historical-data-table"}, flavor=["lxml", "bs4"])[0]  

    df.columns = column
    correct_date_format(df)
    df.sort_values(by=[df.columns[0]], inplace=True)
    df.index = np.arange(0, len(df))  # 일련 번호 오름차순으로 재 설정

    return df

In [11]:
def create_directory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [12]:
def make_pickle(df, pkl_name):
    pkl_directory = '../data/pkl/'
    try:
        if not os.path.exists(pkl_directory):
            os.makedirs(pkl_directory)
    except OSError:
        print("Error: Failed to create the directory.")

    # 데이터 저장: ../data/spx.pkl
    df.to_pickle(pkl_directory+pkl_name)

In [13]:
def read_pickle(pkl_name):
# 데이터 로드
    pkl_directory = '../data/pkl/'
    df = pd.read_pickle(pkl_directory+pkl_name)
    
    return df

[WDM] - Downloading: 100%|██████████| 6.58M/6.58M [00:20<00:00, 11.9MB/s]

In [14]:
spx_url = 'https://kr.investing.com/indices/us-spx-500-historical-data'
spx = ['date', 'spx', 'open', 'high', 'low', 'volume', 'spx_cr']
pkl_name = 'spx.pkl'
df = get_data(spx_url,spx)

# df.set_index('date', inplace=True) # 날짜로 index 만들기
# df.reset_index(inplace=True) # date를 column으로 옮기고, 숫자로 순서대로 index 만들기

make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [15]:
dji_url = 'https://kr.investing.com/indices/us-30-historical-data'
dji = ['date', 'dji', 'open', 'high', 'low', 'volume', 'dji_cr']
pkl_name = 'dji.pkl'
df = get_data(dji_url,dji)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [17]:
nas_url = 'https://kr.investing.com/indices/nasdaq-composite-historical-data'
ixic = ['date', 'ixic', 'open', 'high', 'low', 'volume', 'ixic_cr']
pkl_name = 'nas.pkl'
df = get_data(nas_url,ixic)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [18]:
kospi_url = 'https://kr.investing.com/indices/kospi-historical-data'
kospi = ['date', 'kospi', 'open', 'high', 'low', 'volume', 'kospi_cr']
pkl_name = 'kospi.pkl'
df = get_data(kospi_url,kospi)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [19]:
kosdaq_url = 'https://kr.investing.com/indices/kosdaq-historical-data'
kosdaq = ['date', 'kosdaq', 'open', 'high', 'low', 'volume', 'kosdaq_cr']
pkl_name = 'kosdaq.pkl'
df = get_data(kosdaq_url,kosdaq)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [20]:
sox_url = 'https://kr.investing.com/indices/phlx-semiconductor-historical-data'
sox = ['date', 'sox', 'open', 'high', 'low', 'volume', 'sox_cr']
pkl_name = 'sox.pkl'
df = get_data(sox_url,sox)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [21]:
vix_url = 'https://kr.investing.com/indices/volatility-s-p-500-historical-data'
vix = ['date', 'vix', 'open', 'high', 'low', 'volume', 'vix_cr']
pkl_name = 'vix.pkl'
df = get_data(vix_url,vix)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [22]:
us_bond_10yr_url = 'https://kr.investing.com/rates-bonds/u.s.-10-year-bond-yield-historical-data'
us_10yr = ['date', 'bond_usa_10', 'open', 'high', 'low', 'bond_usa_10_cr']
pkl_name = 'us_10yr_bond.pkl'
df = get_data(us_bond_10yr_url,us_10yr)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [23]:
us_bond_2yr_url = 'https://kr.investing.com/rates-bonds/u.s.-2-year-bond-yield-historical-data'
us_2yr = ['date', 'bond_usa_2', 'open', 'high', 'low', 'bond_usa_2_cr']
pkl_name = 'us_2yr_bond.pkl'
df = get_data(us_bond_2yr_url,us_2yr)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [24]:
us_bond_3mon_url = 'https://kr.investing.com/rates-bonds/u.s.-3-month-bond-yield-historical-data'
us_3mon = ['date', 'bond_usa_3m', 'open', 'high', 'low', 'bond_usa_3m_cr']
pkl_name = 'us_3mon_bond.pkl'
df = get_data(us_bond_3mon_url,us_3mon)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [25]:
kor_bond_10yr_url = 'https://kr.investing.com/rates-bonds/south-korea-10-year-bond-yield-historical-data'
kor_10yr = ['date', 'bond_kor_10', 'open', 'high', 'low', 'bond_kor_10_cr']
pkl_name = 'kor_10yr_bond.pkl'
df = get_data(kor_bond_10yr_url,kor_10yr)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [26]:
kor_bond_2yr_url = 'https://kr.investing.com/rates-bonds/south-korea-2-year-bond-yield-historical-data'
kor_2yr = ['date', 'bond_kor_2', 'open', 'high', 'low','bond_kor_2_cr']
pkl_name = 'kor_2yr_bond.pkl'
df = get_data(kor_bond_2yr_url,kor_2yr)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [28]:
krw_rate_url = 'https://kr.investing.com/currencies/usd-krw-historical-data'
krw_rate = ['date', 'krw', 'open', 'high', 'low', 'vol', 'krw_cr']
pkl_name = 'krw_rate.pkl'
df = get_data(krw_rate_url,krw_rate)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [29]:
nas_futures_url = 'https://kr.investing.com/indices/nq-100-futures-historical-data'
ixic_future = ['date', 'ixic_f', 'open', 'high', 'low', 'volume', 'ixic_f_cr']
pkl_name = 'ixic_future.pkl'
df = get_data(nas_futures_url,ixic_future)
make_pickle(pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [30]:
snp_futures_url = 'https://kr.investing.com/indices/us-spx-500-futures-historical-data'
snp_future = ['date', 'spx_f', 'open', 'high', 'low', 'volume', 'spx_f_cr']
pkl_name = 'snp_future.pkl'
df = get_data(snp_futures_url,snp_future)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [31]:
dow_futures_url = 'https://kr.investing.com/indices/us-30-futures-historical-data'
dji_future = ['date', 'dji_f', 'open', 'high', 'low', 'volume', 'dji_f_cr']
pkl_name = 'dji_future.pkl'
df = get_data(dow_futures_url,dji_future)
make_pickle(pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [32]:
wti_futures_url = 'https://kr.investing.com/commodities/crude-oil-historical-data'
wti_future = ['date', 'wti', 'open', 'high', 'low', 'volume', 'wti_cr']
pkl_name = 'wti_future.pkl'
df = get_data(wti_futures_url,wti_future)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


In [33]:
dollar_index_url = 'https://kr.investing.com/currencies/us-dollar-index-historical-data'
dxy_future = ['date', 'dxy', 'open', 'high', 'low', 'volume', 'dxy_cr']
pkl_name = 'dxy_future.pkl'
df = get_data(dollar_index_url,dxy_future)
make_pickle(df, pkl_name)

임시로 진행함. 날짜 지정하고 적용하기 진행했으면 리턴하세요


#### get  cpi

In [34]:
cpi_url = 'https://kr.investing.com/economic-calendar/cpi-733'
driver.get(cpi_url)

for _ in range(6):
    driver.find_element(By.CSS_SELECTOR, '#showMoreHistory733 > a').send_keys(Keys.ENTER)
    time.sleep(1)
    
df = pd.read_html(driver.page_source, flavor=["lxml", "bs4"])[0]
df.columns = ['date', 'time', 'cpi', 'cpi_anticipated', 'cpi_previous', 'none']

df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())
df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x[:13], "%Y년 %m월 %d일"))

df = df[['date', 'cpi', 'cpi_anticipated', 'cpi_previous']]

df.sort_values(by=[df.columns[0]], inplace=True)
df.index = np.arange(0, len(df))  # 일련 번호 오름차순으로 재 설정

pkl_name = 'cpi.pkl'
make_pickle(df, pkl_name)

#### get append fear and greed

##### https://github.com/hackingthemarkets/sentiment-fear-and-greed/tree/master/datasets
2020년 9월 18일까지 데이터 보관. 이후 다시 취득해야 함.

In [52]:
source_url = 'https://raw.githubusercontent.com/hackingthemarkets/sentiment-fear-and-greed/master/datasets/fear-greed.csv'

In [58]:
df =  pd.read_csv(source_url)[['Date','Fear Greed']]

In [59]:
df.tail()

,Date,Fear Greed
2440,2020-09-14,59
2441,2020-09-15,59
2442,2020-09-16,56
2443,2020-09-17,53
2444,2020-09-18,52


In [64]:
pkl_name = 'fear_greed.pkl'
make_pickle(df, pkl_name)

In [65]:
df1= read_pickle(pkl_name)

In [67]:
df1.tail()

,Date,Fear Greed
2440,2020-09-14,59
2441,2020-09-15,59
2442,2020-09-16,56
2443,2020-09-17,53
2444,2020-09-18,52


In [50]:
import pytz, json

In [51]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) \
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [77]:
BASE_URL = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
START_DATE = '2021-01-01'  # no date before before 2020-08-01, error generated
START_DATE = '2020-09-01'  # no date before before 2020-08-01, error generated

In [69]:
def transform_timezone(x):
    dt = datetime.datetime.fromtimestamp(x / 1000, tz=pytz.utc)
#     tzone = pytz.timezone('US/Eastern')
    tzone = pytz.timezone('Asia/Seoul')
    loc_dt = dt.astimezone(tzone)
    return loc_dt.date()

In [78]:
r = requests.get("{}/{}".format(BASE_URL, START_DATE), headers=headers)
data = r.json()
fg_data = data['fear_and_greed_historical']['data']

In [79]:
df = pd.DataFrame(fg_data)
df.columns = ['date', 'f_g_index', 'rating']
# df.head()

In [80]:
df['date'] = df['date'].apply(lambda x: transform_timezone(x))
df['f_g_index'] = df['f_g_index'].astype('int')

#### get gold price

In [85]:
gold_url = 'https://www.usagold.com/daily-gold-price-history/'
driver.get(gold_url)

In [90]:
def get_gold_data():
    df_append = None
    for yr in ['2020', '2021', '2022', '2023']:

        driver.find_element(By.CSS_SELECTOR, '#goldpricemenu > button').send_keys(Keys.ENTER)
        time.sleep(1)
        driver.find_element(By.LINK_TEXT, yr).send_keys(Keys.ENTER)
        time.sleep(2)
        
        df = pd.read_html(driver.page_source, flavor=["lxml", "bs4"])
        df = df[0].drop(0) # delete empty first row
        df.columns = ['date', 'gold']
        df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%d %b %Y"))

        if df_append is None:
            df_append = df.copy()
            continue
        df_append = pd.concat([df_append, df], ignore_index=True)
#         df_append = df_append.append(df, ignore_index=True)
            
    df_append.sort_values(by=[df_append.columns[0]], inplace=True)
    df_append.drop_duplicates(subset=['date'], inplace=True)
    df_append.index = np.arange(0, len(df_append))
    
    return df_append

In [91]:
df = get_gold_data()

In [97]:
pkl_name = 'gold.pkl'
make_pickle(df, pkl_name)

### fed 금리 get

In [14]:
interest_url = 'https://www.investing.com/economic-calendar/interest-rate-decision-168/'
driver.get(interest_url)

In [15]:
for _ in range(6):
    driver.find_element(By.CSS_SELECTOR, '#showMoreHistory168 > a').send_keys(Keys.ENTER)
    time.sleep(1)

In [16]:
df = pd.read_html(driver.page_source, attrs ={'id':'eventHistoryTable168'}, flavor=["lxml", "bs4"])[0]

In [17]:
df.columns = ['date', 'time', 'fed_rate', 'fed_rate_fore', 'fed_rate_prev', 'none']
# df.replace(np.nan, '', inplace=True)

In [18]:
df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x[:12], "%b %d, %Y"))
df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())
df.drop_duplicates(subset=['date'], keep='last', inplace=True)

In [19]:
df.sort_values(by=[df.columns[0]], inplace=True)
df.index = np.arange(0, len(df))  # 일련 번호 오름차순으로 재 설정

In [20]:
# df = df[df['fed_rate'] != ''] # 데이터가 빈 index는 삭제

In [21]:
pkl_name = 'fed_rate.pkl'
make_pickle(df, pkl_name)

### 한국은행 금리 get append

In [149]:
kor_url = 'https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643'
driver.get(kor_url)

In [150]:
df = pd.read_html(driver.page_source, attrs = {'class': 'fixed'}, flavor=["lxml", "bs4"])[0]

df.columns=  ['release_yr', 'release_date', 'bok_rate']
df_date_temp = df['release_yr'].astype('str')+df['release_date']
df['date'] = df_date_temp.apply(lambda x : datetime.datetime.strptime(x, "%Y%m월 %d일"))
df.drop_duplicates(subset=['date'], keep='last', inplace=True)

df = df[['date', 'bok_rate']] # leave only valid columns

In [154]:
df.sort_values(by=[df.columns[0]], inplace=True)
df.index = np.arange(0, len(df))  # 일련 번호 오름차순으로 재 설정

In [155]:
pkl_name = 'bok_rate.pkl'
make_pickle(df, pkl_name)

### close window

In [22]:
driver.close()
driver.quit()